# Valutazione del dataset

Verrà eseguita la valutazione sia del dataset senza PE, sia con quello PE.

In [1]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import spacy

In [20]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset = []

sentences_en = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_en.csv', header=None, delimiter='aaaaaaaa', names=[0])

i = 0

for sentence in sentences_en[0]:
    current_lid = ""
    current_en_sentence = ""
    i += 1
    print(i)

    for dataset_type in dataset_types:
        if current_lid == "":
            for triple_number in triple_numbers:
                path = "..\\webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
                file_names = []

                for file_name in os.listdir(path):
                    if os.path.isfile(os.path.join(path, file_name)):
                        url = os.path.join(path, file_name)
                        tree = ET.parse(url)
                        root = tree.getroot()

                        for entry in root.iter('entry'):
                            if current_lid == "":
                                for lex in entry.iter('lex'):
                                    if lex.get('lang') == 'en':
                                        if lex.text == sentence:
                                            current_lid = lex.attrib['lid']
                                            current_en_sentence = lex.text

                                if current_lid != "":
                                    for lex in entry.iter('lex'):
                                        if lex.get('lang') == 'it-PE':
                                            if lex.attrib['lid'] == current_lid:
                                                dataset.append(str(i) + " " + lex.text)
                                                print(current_en_sentence)
                                                print(lex.text)
                                                print('\n')
                                                break
                                    

dataset = np.array(dataset)

C:\Users\OliverioM\AppData\Local\Temp\ipykernel_13368\2018346920.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sentences_en = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_en.csv', header=None, delimiter='aaaaaaaa', names=[0])


1
The Aarhus is the airport of Aarhus, Denmark.
L'Aarhus è l'aeroporto di Aarhus, Danimarca.


2
British people is a demonym for people in the United Kingdom.
Britannici è un demonimo che indica gli abitanti del Regno Unito.


3
4
The apoapsis of (410777) 2009 FD is 259776702.47055 kilometers.
L'apopsi di (410777) 2009 FD è di 259776702,47055 chilometri.


5
The main ingredients of Ajoblanco are bread, almonds, garlic, water, olive oil.
Gli ingredienti principali dell'Ajoblanco sono pane, mandorle, aglio, acqua e olio d'oliva.


6
The School of Business and Social Sciences at the Aarhus University has academic staff size of 737.
La Scuola di Business e Scienze Sociali dell'Università di Aarhus ha un organico accademico di 737 persone.


7
8
9
10
The International Tennis Federation, which is the sport's governing body, has agreed to let Acharya Institute of Technology offer tennis at the school.
La Federazione Internazionale di Tennis, che è l'organo di governo di questo sport, ha accet

In [23]:
dataset_copy = dataset.copy()
#rimuovi i primi 4 caratteri di ogni riga
dataset_copy = np.char.lstrip(dataset_copy, '1234567890 ')

dataset_copy

array(["L'Aarhus è l'aeroporto di Aarhus, Danimarca.",
       'Britannici è un demonimo che indica gli abitanti del Regno Unito.',
       "L'apopsi di (410777) 2009 FD è di 259776702,47055 chilometri.",
       "Gli ingredienti principali dell'Ajoblanco sono pane, mandorle, aglio, acqua e olio d'oliva.",
       "La Scuola di Business e Scienze Sociali dell'Università di Aarhus ha un organico accademico di 737 persone.",
       "La Federazione Internazionale di Tennis, che è l'organo di governo di questo sport, ha accettato di permettere all'Acharya Institute of Technology di offrire il tennis nella scuola.",
       'Alan Shepard morì il 21 luglio 1998 in California (il cui senatore era Dianne Feinstein).',
       'Il musicista black metal Aaron Turner, che suona la chitarra elettrica, ha fatto parte dei gruppi Twilight e Greymachine. Il black metal ha una fusione musicale chiamata Death metal.',
       'Il cantante solista Ace Wilder è nato il 23 luglio 1982 in Svezia.',
       "Abraham

In [24]:
dt_dataset = pd.DataFrame(dataset_copy)
dt_dataset.to_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl_PE.csv', index=False, header=False)

## Valutazione

In [91]:
#utils
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

#blue 
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

#meteor
from nltk.translate.meteor_score import meteor_score

#pvalue
from scipy.stats import ttest_ind

df_manual = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_manual.csv', header=None, delimiter='aaaaaaaa', names=[0])
df_deepl = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl.csv', header=None, delimiter='aaaaaaaa', names=[0])
df_deepl_PE = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl_PE.csv', header=None, delimiter='aaaaaaaa', names=[0])

C:\Users\OliverioM\AppData\Local\Temp\ipykernel_13368\3478680711.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_manual = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_manual.csv', header=None, delimiter='aaaaaaaa', names=[0])
C:\Users\OliverioM\AppData\Local\Temp\ipykernel_13368\3478680711.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_deepl = pd.read_csv('../1_Valutazione modelli/confonto_datasets/sentences_it_deepl.csv', header=None, delimiter='aaaaaaaa', names=[0])
C:\Users\OliverioM\AppData\Local\Temp\ipykernel_13368\3478680711.py:18: ParserWarning: Falling b

### BLEU

In [67]:
def blue_evaluation(df):
    blue_scores = []
    
    for i in range(len(df)):
        reference = df_manual[0][i]
        candidate = df[0][i]

        #remove " from candidate
        candidate = candidate.replace('"', '')

        blue_score = sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
        blue_scores.append(blue_score)
        
    return blue_scores

In [92]:
print('BLUE SCORE \n')

df_deepl['bleu_score'] = blue_evaluation(df_deepl)
print('- DeepL: ', df_deepl['bleu_score'].mean())

df_deepl_PE['bleu_score'] = blue_evaluation(df_deepl_PE)
print('- DeepL PE: ', df_deepl_PE['bleu_score'].mean())

BLUE SCORE 

- DeepL:  0.8645016411427956
- DeepL PE:  0.8739608306066333


### METEOR

In [69]:
def meteor_evaluation(df):
    meteor_scores = []
    
    for i in range(len(df)):
        reference = [word_tokenize(df_manual[0][i])]
        candidate = word_tokenize(df[0][i])

        candidate = [word.replace('"', '') for word in candidate]

        meteor_scores.append(meteor_score(reference, candidate))
        
    return meteor_scores

In [93]:
print('\nMETEOR SCORE \n')

df_deepl['meteor_score'] = meteor_evaluation(df_deepl)
print('- DeepL: ', df_deepl['meteor_score'].mean())

df_deepl_PE['meteor_score'] = meteor_evaluation(df_deepl_PE)
print('- DeepL PE: ', df_deepl_PE['meteor_score'].mean())



METEOR SCORE 

- DeepL:  0.8492233619783042
- DeepL PE:  0.8586311906894882
